# Find the seed of an existing image

### Imports

In [ ]:
import sys
sys.path.append('../')

import os
import torch
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
from scipy.interpolate import interp1d

from training.generator import Generator
from training import utils
from training.settings import *

utils.reset_rand()

### Check GPU

In [ ]:
utils.check_gpu()

### Import the model

In [ ]:
MODEL_PATH = 'models/faces_256.pt'

model = Generator().to(DEVICE)
model.load_state_dict(torch.load(os.path.join('..', MODEL_PATH)))
model.eval()
model.requires_grad_(False)

### Mean W image

In [ ]:
image = model.generate_one(0.0)

plt.imshow(image)
plt.axis('off')
plt.show()

### Settings

In [ ]:
IMAGE_PATH = ''			# Path to the image to project
SAVE_DIR = None			# Directory to save the images to
NB_STEPS = 1_000		# Number of steps to use for projection
MAX_LEARNING_RATE = 0.1	# Maximum learning rate to use
WARMUP_STEPS = 50		# Number of steps to use for warmup
COOLDOWN_STEPS = 750	# Number of steps to use for cooldown
NOISE_STEPS = 750		# Number of steps using noise on W
NOISE_SCALE = 0.05		# Noise scale